# EDA: source 

## Set up

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.2.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b090d6a2-9f74-4335-a817-acac08ccd4d9;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 347ms :: artifacts dl 22ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

3.2.0


In [4]:
## other libs
from pyspark.sql.functions import col, udf, count, sum, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
from urllib.parse import urlparse

## Read In data

In [5]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_comments = "project/submissions/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
print(f"reading comments from {s3_path}")
posts = spark.read.parquet(s3_path, header=True)
print(f"shape of the posts dataframe is {posts.count():,}x{len(posts.columns)}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://sagemaker-us-east-1-165729782536/project/submissions/yyyy=*


24/04/03 12:52:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/04/03 12:52:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


shape of the posts dataframe is 1,847,138x68
CPU times: user 2.98 s, sys: 731 ms, total: 3.71 s
Wall time: 6min 21s


### create the source column

In [6]:
def extract_refined_domain(url):
    try:
        parsed_url = urlparse(url)
        # Handling Google AMP URLs
        if "www.google.com" in parsed_url.netloc in url:
            # amp_url = url.split("/amp/s/")
            if 'amp/s/' in url:
                amp_url =  url.split('amp/s/')[1]
            elif 'url=' in url:
                amp_url=  url.split('url=')[1]
            domain = urlparse('http://' + amp_url).netloc
        else:
            domain = parsed_url.netloc

        # Splitting the domain and returning the first meaningful part
        domain_parts = domain.split('.')
        domain_part = domain_parts[1] if domain_parts[0] in ['www', 'amp'] else domain_parts[0]


        # For Twitter, returning the username
        if "twitter.com" in domain:
            path_parts = parsed_url.path.split('/')
            return path_parts[1] if len(path_parts) > 1 else domain_part

        return domain_part.lower()
    except:
        return None
    

extract_refined_domain_udf = udf(extract_refined_domain, StringType())

posts = posts.withColumn("source", extract_refined_domain_udf("url"))
# filter out source=reddit, which mean the post is deleted
posts = posts.filter((col("source") != "reddit") &(col("source") != ""))
posts = posts.filter((col("url") != "") & (col("url").isNotNull()))
posts = posts.select("title", "url", "subreddit", "score", "source", "created_utc", "num_comments")

print(f"shape of the posts dataframe is {posts.count():,}x{len(posts.columns)}")
posts.show(5, truncate=False)

shape of the posts dataframe is 1,753,936x7


+----------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-----+----------+-------------------+------------+
|title                                                                             |url                                                                                                                                                                                                                      |subreddit|score|source    |created_utc        |num_comments|
+----------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/conda/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/conda/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/conda/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                              
  File "/opt/conda/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/conda/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/conda/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/p

### top table of all

In [7]:
# Aggregate by 'source'
agg_df = posts.groupBy('source').agg(
    count('source').alias('frequency'), 
    sum('score').alias('total_score')
)

# Order by frequency and total_score in descending order and take the top 100
top_agg_df = agg_df.orderBy(col('frequency').desc(), col('total_score').desc()).limit(100)


In [8]:
# Path for storing the CSV
csv_path = '../data/csv/top100_source.csv'

# Store the DataFrame in CSV format
top_agg_df.write.csv(csv_path, header=True, mode="overwrite")

### top table for each subreddit

In [ ]:
agg_df = posts.groupBy('source', 'subreddit').agg(
    count('source').alias('frequency'), 
    sum('score').alias('total_score')
)

# Order by frequency in descending order and take the top results
top_agg_df = agg_df.orderBy('frequency', ascending=False)

# Show the results
top_agg_df.show(truncate=False)

In [ ]:
print(f"shape of the agg_df dataframe is {agg_df.count():,}x{len(agg_df.columns)}")

In [ ]:
agg_df.orderBy('total_score', ascending=False).show()

In [ ]:
# Define a window specification
windowSpec = Window.partitionBy("subreddit").orderBy(col("frequency").desc(), col("total_score").desc())

# Rank each row within its subreddit based on frequency and score
ranked_df = agg_df.withColumn("rank", row_number().over(windowSpec))

# Filter for the top 20 in each subreddit
top_100_each_subreddit = ranked_df.filter(col("rank") <= 100)

# Show the results
top_100_each_subreddit.show(truncate=False)

In [ ]:
print(f"shape of the agg_df dataframe is {top_100_each_subreddit.count():,}x{len(top_100_each_subreddit.columns)}")

In [ ]:
top_100_each_subreddit.coalesce(1).write.csv("top100source.csv", header=True, mode="overwrite")

In [ ]:
def extract_refined_domain(url):
    try:
        parsed_url = urlparse(url)
        # Handling Google AMP URLs
        if "www.google.com" in parsed_url.netloc in url:
            amp_url = url.split("/amp/s/")[1]
            domain = urlparse(amp_url).netloc
            print(domain)
        else:
            domain = parsed_url.netloc

        # Splitting the domain and returning the first meaningful part
        domain_parts = domain.split('.')
        domain_part = domain_parts[0] if domain_parts[0] != 'www' else domain_parts[1]

        # For Twitter, returning the username
        if "twitter.com" in domain:
            path_parts = parsed_url.path.split('/')
            return path_parts[1] if len(path_parts) > 1 else domain_part

        return domain_part
    except:
        return None


# extract_refined_domain_udf = udf(extract_refined_domain, StringType())

# posts = posts.withColumn("source", extract_refined_domain_udf("url"))
# # filter out source=reddit, which mean the post is deleted
# posts = posts.filter(col("source") != "reddit")

# posts.select("url", "source").show(truncate=False)